In [1]:
import pandas as pd
import numpy as np

from bea_downloader import BEADownloader

In [2]:
bd = BEADownloader('3E7CB4CE-838D-4BB0-AE7C-9BACB931D976')

# BEA

Here I get the required tables from BEA, and add some classification codes from the replication stata file.
After that I will compute some variables

## Fixed assets

Here a combination of tables (see code bellow), and concatenate them, and transform them to a long level (to add stuff later)

In [3]:
# Read in stat dict
bea_key = pd.read_stata('Data/Temp/levelkey_bea.dta')

# Generate a list of tables to read and their corresponding variable names
table_num = {
    '301': 'kp_',
    '302': 'kq_',
    '304': 'depp_',
    '305': 'depq_',
    '307': 'ip_',
    '308': 'iq_',
    '309': 'agep_'
}

table_suff = {
    'E': 'eq_',
    'S': 'st_',
    'I': 'ip_',
    'ESI': 'all_',
}

table_names = {}
for tnn, tnl in table_num.items():
    for tsn, tsl in table_suff.items():
        table_names[tnn + tsn] = 'a1_' + tnl + tsl + 'bea'
        
# Download all tables
tables = []
for table_name, variable_label in table_names.items():
    table = bd.get_table('FAAt' + table_name, frequency = 'A', 
                         database = 'FixedAssets').reset_index()
    
    table['var_name'] = variable_label
    tables.append(table)

# Concatenate the tables 
data_fa = pd.concat(tables).reset_index(drop = True)

# Create indcode and year, drop date, rename LineDescription
data_fa['year'] = data_fa.date.dt.year
data_fa['indcode'] = data_fa.SeriesCode.str.slice(3,7)
data_fa.drop(columns = ['date'], inplace = True)
data_fa.rename(columns = {'LineDescription': 'ind'}, inplace = True)

In [4]:
# To long format
data_fa = data_fa.pivot_table(index = ['year', 'ind', 'indcode'], 
                    columns = ['var_name'])
data_fa.columns = data_fa.columns.droplevel(0)
data_fa = data_fa.reset_index()

## Value added

Here I use a merge of two downloaded excel tables (because API is too wierd). Note that on the BEA site it says that the date pre 1997 is not yet compatible with data post 1997, the authors seem to have ignored that.

The tables are downloaded from 
- GDP By Industry -> Historical 1947-1997 data -> Annual zip tables
- GDP By Industry -> Begin using data -> Annual zip tables

I do some reshaping to get data in a semi-long format: so industry and year on the x axis, and type of value added (GOS, wages, taxes) on the y axis

In [5]:
# Helper function to tidy data
def tidy_va_data(data):
    data.rename(columns = {'Unnamed: 1': 'ind'}, inplace = True)

    # Replace some values
    data.loc[data.ind.str.contains('Compensation'), 'ind'] = 'Wages'
    data.loc[data.ind.str.contains('Taxes'), 'ind'] = 'Taxes'
    data.loc[data.ind.str.contains('operating'), 'ind'] = 'GOS'
    data['ind'] = data.ind.str.strip()

    # Create a new column, modify name column
    data['type'] = data.loc[data.ind.isin(['Wages', 'Taxes', 'GOS']), 'ind']
    data.loc[data['type'].notna(), 'ind'] = None
    data['ind'] = data.ind.fillna(method = 'ffill')
    data['type'] = data['type'].fillna('Total')

    # Table to long, and then to semi-long
    data = data.melt(id_vars = ['ind', 'type'], var_name = 'year')
    data['value'] = pd.to_numeric(data.value, errors = 'coerce')

    # Get data to billions of dollars (to match FA)
    data['value'] = data['value']/1000
    
    data = data.pivot_table(index = ['ind', 'year'], columns = ['type'], 
                              values = 'value').reset_index()
    
    # Year to numeric
    data['year'] = pd.to_numeric(data.year)

    return data

In [6]:
# Load and tidy data
data_va = pd.read_excel('Data/Raw inputs/ValueAdded.xlsx', sheet_name = 'Components',
                        skiprows = 5, skipfooter = 17).set_index('Line')
data_va_h = pd.read_excel('Data/Raw inputs/ValueAdded_H.xlsx', sheet_name = 'Components',
                        skiprows = 5, skipfooter = 17).set_index('Line')

data_va = tidy_va_data(data_va)
data_va_h = tidy_va_data(data_va_h)

# Put them together
data_va = pd.concat([data_va, data_va_h])
data_va = data_va.sort_values(['ind', 'year']).reset_index(drop = True)

# Drop total, rename the rest
data_va.drop(columns = 'Total', inplace = True)
data_va.rename(columns = {'Taxes': 'a1_taxes_bea',
                          'Wages': 'a1_wages_bea',
                          'GOS': 'a1_gos_bea'}, inplace = True)

## Gross output

The datasets come in the zipfiles from Value Added section. Analysis is simpler, there is only one value column, so no extensive reshaping needed.

In [7]:
# Helper function to tidy data
def tidy_go_data(data):
    data.rename(columns = {'Unnamed: 1': 'ind'}, inplace = True)
    data['ind'] = data.ind.str.strip()

    # Table to long
    data = data.melt(id_vars = ['ind'], var_name = 'year')
    data['value'] = pd.to_numeric(data.value, errors = 'coerce')
    
    # Get data to billions of dollars (to match FA)
    data['value'] = data['value']/1000
    
    data.rename(columns = {'value': 'a1_output_bea'}, inplace = True)
    
    # Year to numeric
    data['year'] = pd.to_numeric(data.year)
    
    # Drop if year les or equal 1963
    data = data.query('year > 1963')

    return data

In [8]:
# Load and tidy data
data_go = pd.read_excel('Data/Raw inputs/GrossOutput.xlsx', sheet_name = 'GO',
                        skiprows = 5, skipfooter = 8).set_index('Line')
data_go_h = pd.read_excel('Data/Raw inputs/GrossOutput_H.xlsx', sheet_name = 'GO',
                        skiprows = 5, skipfooter = 9).set_index('Line')

In [9]:
data_go = tidy_go_data(data_go)
data_go_h = tidy_go_data(data_go_h)

# Put them together
data_go = pd.concat([data_go, data_go_h])
data_go = data_go.sort_values(['ind', 'year']).reset_index(drop = True)

## Merge data

Here I merge the three datasets, and add some variables from a stata matching file provided by the authors.

When merging the key with the datasets, I perform an inner merge, which automatically throws away those industries we don't care about.

In [10]:
# Get matching file, fix names (excel...)
bea_key = pd.read_stata('Data/Temp/levelkey_bea.dta')
bea_key['ind'] = bea_key.ind.str.replace(r'\\\d\\', '').str.strip()

In [11]:
# Merge FA and VA data
data = data_fa.merge(data_va, on = ['ind', 'year'], how = 'outer')

# Merge GO data
data = data.merge(data_go, on = ['ind', 'year'], how = 'outer')

# Merge FA data with key
data = bea_key.merge(data, on = 'ind', how = 'inner')

# Extend indcode
ind = data.query('~indcode.isna()').groupby(['ind', 'indcode']).size()
ind = ind.reset_index().drop(columns = [0])

data = data.drop(columns = ['indcode']).merge(ind, on = 'ind')

## Normalizations and aggregations

Here I create some pivoting/aggregation
- I make FA quantity variables real, taking 2009 as the base. Note that in the BEA database, the base is set as 2012, so I do some adjustments.
- After that, I create a pivot table, summing accross `ind_short` and year, I also keep the `siccode` and discard the other descriptives.
- During the course of the aggregations, some nan variables were transformed to 0, reverse that

In [12]:
# Create agg variables, normalized to 2009
p_vars = []
q_vars = []

for i in ['k', 'i', 'dep']:
    for j in ['all_', 'eq_', 'st_', 'ip_']:
        p_vars.append('a1_' + i + 'p_' + j + 'bea')
        q_vars.append('a1_' + i + 'q_' + j + 'bea')

vp_2009 = data.query('year == 2009')[['indcode'] + p_vars].set_index('indcode')
vq_2009 = data.query('year == 2009')[['indcode'] + q_vars].set_index('indcode')
vp_2009.columns = q_vars

div_2009 = lambda x: x.multiply(vp_2009.loc[x.name,:]/100)\
                      .divide(vq_2009.loc[x.name,:]/100)\
                      .replace([np.inf], np.nan)

data.loc[:, q_vars] = data.groupby('indcode')[q_vars].apply(div_2009)

# Kick out some variables, create pivot table
data.drop(columns = ['indcode', 'ind', 'keep_ind', 'beacode'], inplace = True)

data = data.melt(id_vars = ['year', 'ind_short', 'siccode'])\
           .groupby(['year', 'ind_short', 'variable'])\
           .agg({'value': [np.sum], 'siccode': [np.mean]})

data.columns = data.columns.droplevel(1)
data = data.set_index('siccode', append = True).unstack(2)
data.columns = data.columns.droplevel(0)
data = data.reset_index()

# Reverse nans
fa_cols = list(table_names.values())
va_cols = ['a1_taxes_bea', 'a1_wages_bea', 'a1_gos_bea']
go_cols = ['a1_output_bea']

fa_m = data_fa.year.min(); fa_M = data_fa.year.max()
va_m = data_va.year.min(); va_M = data_va.year.max()
go_m = data_go.year.min(); go_M = data_go.year.max()

data.loc[~data.year.between(fa_m, fa_M), fa_cols] = None
data.loc[~data.year.between(va_m, va_M), va_cols] = None
data.loc[~data.year.between(go_m, go_M), go_cols] = None

## New variables

Here I create some new variables:

### Industry level
- `a1_os_bea`: operatirng surplus (GOS - depreciation)
- Net investment, investment rates and depreciation rates

In [13]:
# For this to work we need to order data
data.sort_values(['ind_short', 'year'], inplace = True)

# Operating surplus
data['a1_os_bea'] = data['a1_gos_bea'] - data['a1_depp_all_bea']

# Generate new var names
for i in ['all', 'eq', 'st', 'ip']:
    var_defs = f'''
        a1_depk_{i}_bea = a1_depp_{i}_bea/a1_kp_{i}_bea.shift(1)
        a1_nip_{i}_bea = a1_ip_{i}_bea - a1_depp_{i}_bea
        a1_ik_{i}_bea = a1_ip_{i}_bea/a1_kp_{i}_bea.shift(1)
        a1_nik_{i}_bea = a1_nip_{i}_bea/a1_kp_{i}_bea.shift(1)
        a1_dkp_{i}_bea = a1_kp_{i}_bea - a1_kp_{i}_bea.shift(1)
        
        a1_iy_{i}_bea = a1_ip_{i}_bea/a1_os_bea
        a1_niy_{i}_bea = a1_nip_{i}_bea/a1_os_bea
        a1_nigy_{i}_bea = a1_ip_{i}_bea/a1_gos_bea
        
        a1_depkq_{i}_bea = a1_depq_{i}_bea/a1_kq_{i}_bea.shift(1)
        a1_niq_{i}_bea = a1_iq_{i}_bea/a1_depq_{i}_bea.shift(1)
        a1_nikq_{i}_bea = a1_niq_{i}_bea/a1_kq_{i}_bea.shift(1)
        a1_dkq_{i}_bea = (a1_kq_{i}_bea - a1_kq_{i}_bea.shift(1))/a1_kq_{i}_bea.shift(1)
    '''
    
    data = data.groupby('ind_short').apply(lambda x: x.eval(var_defs))\
                                    .reset_index(drop = True)\
                                    .replace([np.inf, -np.inf], np.nan)
    
    # Limit some variables
    data[f'a1_nigy_{i}_bea'] = data[f'a1_nigy_{i}_bea'].clip(-0.2, 2)
    data[f'a1_niy_{i}_bea'] = data[f'a1_nigy_{i}_bea'].clip(-0.2, 2).replace(2, np.nan)
    data[f'a1_iy_{i}_bea'] = data[f'a1_nigy_{i}_bea'].clip(-0.2, 2)

# Some other stuff
var_defs = '''
    a1_depk_exip_bea = (a1_depp_all_bea-a1_depp_ip_bea)/(a1_kp_all_bea.shift(1)-a1_kp_ip_bea.shift(1))

    a1_nip_exip_bea = (a1_ip_all_bea-a1_ip_ip_bea) - (a1_depp_all_bea-a1_depp_ip_bea)
    a1_ik_exip_bea = (a1_ip_all_bea-a1_ip_ip_bea) / (a1_kp_all_bea.shift(1)-a1_kp_ip_bea.shift(1))
    a1_nik_exip_bea = a1_nip_exip_bea / (a1_kp_all_bea.shift(1)-a1_kp_ip_bea.shift(1))

    a1_osk_bea = a1_os_bea / a1_kp_all_bea.shift(1)
'''

data = data.groupby('ind_short').apply(lambda x: x.eval(var_defs))\
                                .reset_index(drop = True)\
                                .replace([np.inf, -np.inf], np.nan)

### Aggregate level

Basically the same stuff as above, but on aggregate level (i.e. across all industries for each year)

In [14]:
gb = data.groupby('year')

for i in ['all', 'eq', 'st', 'ip']:
    
    data[f'a_kp_{i}_bea'] = gb[f'a1_kp_{i}_bea'].transform(np.sum)
    data[f'a_ip_{i}_bea'] = gb[f'a1_ip_{i}_bea'].transform(np.sum)
    data[f'a_depp_{i}_bea'] = gb[f'a1_depp_{i}_bea'].transform(np.sum)
    
    data[f'a_kq_{i}_bea'] = gb[f'a1_kq_{i}_bea'].transform(np.sum)
    data[f'a_iq_{i}_bea'] = gb[f'a1_iq_{i}_bea'].transform(np.sum)
    data[f'a_depq_{i}_bea'] = gb[f'a1_depq_{i}_bea'].transform(np.sum)
    
    var_defs = f'''
        a_nip_{i}_bea = a_ip_{i}_bea - a_depp_{i}_bea
        a_ik_{i}_bea = a_ip_{i}_bea/a_kp_{i}_bea.shift(1)
        a_depk_{i}_bea = a_depp_{i}_bea/a_kp_{i}_bea.shift(1)
        a_nik_{i}_bea = a_nip_{i}_bea/a_kp_{i}_bea.shift(1)    

        a_niq_{i}_bea = a_iq_{i}_bea - a_depq_{i}_bea
        a_ikq_{i}_bea = a_iq_{i}_bea/a_kq_{i}_bea.shift(1)
        a_depkq_{i}_bea = a_depq_{i}_bea/a_kq_{i}_bea.shift(1)
        a_nikq_{i}_bea = a_niq_{i}_bea/a_kq_{i}_bea.shift(1)
    '''
    
    data = data.groupby('ind_short').apply(lambda x: x.eval(var_defs))\
                                    .reset_index(drop = True)\
                                    .replace([np.inf, -np.inf], np.nan)

# Some other stuff
var_defs = '''
    a_depk_exip_bea = (a_depp_all_bea-a_depp_ip_bea)/(a_kp_all_bea.shift(1)-a_kp_ip_bea.shift(1))
    a_nip_exip_bea = (a_ip_all_bea-a_ip_ip_bea) - (a_depp_all_bea-a_depp_ip_bea)
    a_ik_exip_bea = (a_ip_all_bea-a_ip_ip_bea) / (a_kp_all_bea.shift(1)-a_kp_ip_bea.shift(1))
    a_nik_exip_bea = a_nip_exip_bea / (a_kp_all_bea.shift(1)-a_kp_ip_bea.shift(1))
'''

data = data.groupby('ind_short').apply(lambda x: x.eval(var_defs))\
                                .reset_index(drop = True)\
                                .replace([np.inf, -np.inf], np.nan)

gb = data.groupby('year')

# Some more other stuff
data['a_gos_bea'] = gb['a1_gos_bea'].transform(np.sum)
data['a_os_bea'] = gb['a1_os_bea'].transform(np.sum)
data['a_output_bea'] = gb['a1_output_bea'].transform(np.sum)

var_defs = '''
    a_osk_bea = a_os_bea/a_kp_all_bea.shift(1)
    a_iy_bea = a_ip_all_bea/a_os_bea
    a_niy_bea = a_nip_all_bea/a_os_bea
    a_nigy_bea = a_nip_all_bea/a_gos_bea
'''

data = data.groupby('ind_short').apply(lambda x: x.eval(var_defs))\
                                .reset_index(drop = True)\
                                .replace([np.inf, -np.inf], np.nan)

In [15]:
# Save data
data.rename(columns = {'ind_short': 'indcode'})\
    .to_stata('Data/Intermediate/BEA_industry.dta', write_index = False)